## Imports and Functions

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache
import lmfit as lm


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt

import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)


colors = pc.qualitative.D3
angstrom = "\u212B"
gamma = "\u0393"

# Load Data

In [ ]:
# ## 10 ML Film (XUV + Lamp) ##

# ## Convert .ibw to .h5 ##
# # ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\Lamp\Swept"  # swept
# ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\Lamp\Fixed"  # fixed

# # Convert ibw to hdf5
# fn = r"Img0_g_kw.ibw"
# HDF5_loader.ibw_to_hdf5(ddir, fn, export=True)

# # Check conversion worked
# data, theta, energy = HDF5_loader.load_hdf5(
#     ddir, r"Img0_g_kw.h5"
# )  # load data from hdf5
# data.shape, theta.shape, energy.shape

In [ ]:
## Load Data ##

ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\Lamp\Swept"  # swept
# file = r"Img0_sweep_kw.h5"
file = r"Img1_avg_g_kw.h5"

# ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\Lamp\Fixed"  # fixed
# file = r"Img0_g_kw.h5"  # lamp band structure

# data1, kx1, energy1 = loading_functions.load_hdf5(ddir, file)
data2, kx2, energy2 = loading_functions.load_hdf5(ddir, file)

In [ ]:
xaxis_title = f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)"
yaxis_title = f"E<sub>K</sub> (eV)"
# title = f""

In [ ]:
## Plot Data ##

x, y, data = kx1, energy1, (data1 + data2)
# x, y, data = kx2, energy2, data2

fig = tr_functions.thesis_fig(
    title=f"{file}",
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
)

fig.add_trace(
    go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(data), coloraxis="coloraxis")
)

fig.show()

## Fit for EF

In [ ]:
x, y, data = kx_5ml, energy_5ml, data_5ml

# xuv
xlim = (-0.5, 0.5)
ylim = (17.5, 18.8)  # EF -- xuv

# lamp
xlim = (-0.73, 0.46)
# ylim = (16.5, 17.5)  # EF -- lamp

new_x, new_y, new_data = analysis_functions.limit_dataset(x, y, data, xlim, ylim)

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=f"EDC of E<sub>F</sub>",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    dtick_y=0.2,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=new_x,
    y=new_y,
    data=analysis_functions.norm_data(new_data),
    ylims=None,
    x_range=None,
)

# Plot Data
fig.add_trace(go.Scatter(x=y_1d, y=col, name="data", line=dict(color=colors[0])))

fig.show()

In [ ]:
## Fit FD ##
import lmfit as lm

T = 10  # measurement temp
k_B = 8.617333e-5  # eV/K

x = y_1d
data = col

offset_type = "constant"


## FD
def fermi_dirac(x, center, theta, amp):
    arg = (x - center) / (2 * theta)  # x=E, center=mu, theta = k_B * T
    return -amp / 2 * np.tanh(arg)


## Offset
c = np.mean(data)
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

full_model = lm.models.Model(fermi_dirac) + offset

params = full_model.make_params()

# params["center"].value = 18.3
params["center"].value = 16.9
params["center"].vary = True
# params["center"].min = 1.85
# params["center"].max = 1.95

params["theta"].value = k_B * T
params["theta"].vary = True


for param in params:
    params[param].value = np.float32(params[param].value)
    params[param].min = np.float32(params[param].min)
    params[param].max = np.float32(params[param].max)
    # print(params[param].value.dtype)


# fit = full_model.fit(data, x=x, params=params, nan_policy="propagate")
fit = full_model.fit(data.astype(np.float32), x=x.astype(np.float32), params=params)

fit.plot()

print(f"EF: {fit.params['center'].value}")

## Fit for HOMO

In [ ]:
EF = 16.9  # lamp

In [ ]:
xaxis_title = f"k<sub>x</sub> ({angstrom}<sup>-1</sup>)"
yaxis_title = f"E - E<sub>F</sub> (eV)"
# yaxis_title = f"E<sub>K</sub> (eV)"
title = f"{file}"
# title = f"5ML C<sub>60</sub> Band Structure<br>(XUV)"
title = f"10ML C<sub>60</sub> Band Structure<br>(He Lamp)"

In [ ]:
x, y, data = kx1, energy1, (data1 + data2)

xlim = (-0.45, 0.52)  # swept
ylim = None  # for EDC

## For fits
# ylim = (12.67, 16.963)  # Ek -- lamp

new_x, new_y, new_data = analysis_functions.limit_dataset(x, y, data, xlim, ylim)

new_y = new_y - EF

In [ ]:
## Plot Data ##

fig = tr_functions.thesis_fig(
    title=title,
    xaxis_title=xaxis_title,
    yaxis_title=yaxis_title,
    equiv_axes=False,
    # height=500,  # xuv
    height=800,  # lamp
    width=600,
    dtick_y=2,
    dtick_x=0.2,
)

fig.add_trace(
    go.Heatmap(
        x=new_x,
        y=new_y,
        z=analysis_functions.norm_data(new_data),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(colorscale="Blues", reversescale=False)
fig.update_yaxes(range=(np.min(-8.8), np.max(new_y)))
# fig.show(renderer="svg")

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    # title=f"EDC of C<sub>60</sub> HOMO and HOMO-1",
    # title=f"1ML C<sub>60</sub> EDC (XUV)",
    title=f"EDC",
    yaxis_title=yaxis_title,
    xaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    # dtick_y=0.2,
    # height=500,  # xuv
    height=800,  # lamp
    width=300,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=new_x,
    y=new_y,
    data=analysis_functions.norm_data(new_data),
    ylims=None,
    x_range=None,
)

# Plot Data
fig.add_trace(go.Scatter(x=col, y=y_1d, name="data", line=dict(color=colors[0])))

fig.update_yaxes(range=(-8.8, np.max(y_1d)))

In [ ]:
## Get and Plot 1D Data ##

fig = tr_functions.thesis_fig(
    # title=f"5ML C<sub>60</sub> EDC (XUV)",
    title=f"10ML C<sub>60</sub> EDC (He Lamp)",
    xaxis_title=yaxis_title,
    yaxis_title="Intensity (arb. u)",
    equiv_axes=False,
    gridlines=False,
    dtick_y=0.2,
)

y_1d, col = tr_functions.get_1d_x_slice(
    x=new_x,
    y=new_y,
    data=analysis_functions.norm_data(new_data),
    ylims=(-4.44, 0.31),
    x_range=None,
)

# Plot Data
fig.add_trace(go.Scatter(x=y_1d, y=col, name="data", line=dict(color=colors[0])))

## For After Fit
fig.add_trace(go.Scatter(x=y_1d, y=fit.eval(x=y_1d), name="fit"))

components = fit.eval_components(x=y_1d)
for model_name, model_value in list(components.items())[0:2]:
    fig.add_annotation(
        x=fit.params[f"{model_name}center"].value,
        y=fit.eval(x=fit.params[f"{model_name}center"].value) + 0.2,
        # xref="x domain",
        # yref="y domain",
        # x=0.01,
        # y=0.99,
        showarrow=False,
        # text=f'Peak center: {fit.params[f"center"].value:.2f} +/- {fit.params[f"center"].stderr:.4f} mm<br>FWHM: {tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params["sigma"].value)):.3f} +/- {tr_functions.mm_to_ps(tr_functions.sig_to_fwhm(fit.params[f"sigma"].stderr)):.4f} ps',
        text=f'Peak center:<br>{fit.params[f"{model_name}center"].value:.2f} +/- {fit.params[f"{model_name}center"].stderr:.4f} eV<br><br>FWHM:<br>{fit.params[f"{model_name}fwhm"].value:.4f} +/- {fit.params[f"{model_name}fwhm"].stderr:.4f} eV',
        font=dict(size=18),
    )

fig.show()

In [ ]:
## Fit Data ##

x = y_1d
data = col

offset_type = "constant"

## Offset
# c = np.mean(data)
c = 0
b = (data[-1] - data[0]) / (x[-1] - x[0])
a = 0

offset = fitting_functions.offset_model(offset_type, a, b, c)

## Gaussian
gauss1 = fitting_functions.make_gaussian(num="A_", amplitude=1, center=-2, sigma=0.5)
gauss2 = fitting_functions.make_gaussian(num="B_", amplitude=1, center=-3.5, sigma=0.5)


full_model = gauss1 + gauss2 + offset

fit = full_model.fit(data, x=x)

fit.plot()

print("HOMO")
print(fit.params["iA__center"])
print(fit.params["iA__fwhm"])
print("HOMO-1")
print(fit.params["iB__center"])
print(fit.params["iB__fwhm"])

# fit.params

In [ ]:
fit.params